## Naive Bayes From Scratch With MNIS dataset Using Python

**Naive Bayes** 알고리즘을 이용해서 손글씨 데이터를 예측해보도록 하겠습니다.

나이브 베이즈(또는 베이지안) 분류(**Naive Bayesian Classification**)은 지도 학습(Supervised Learning)을 사용한 간단한 분류 중 하나입니다. 이 분류는 분류를 위해서 **베이즈 룰(Bayes' Rule)**을 기본적으로 사용하고 있습니다. **베이즈 룰을 사용하는 가장 큰 이유는 조건부 확률을 구할 때 베이즈 룰을 이용 시 더 손쉽게 값을 구하는 경우가 있기 때문입니다.**

쉽게 표현한면, 각각의 데이터들이 주어졌을 때, 해당 데이터가 가리키는 값(라벨)이 A일 확률, B일 확률을 각각 구한 뒤, 가장 확률이 높은 값(라벨)을 채택하는 방식이라고 이해할 수 있습니다.

![NB_img](./img/p_1.png)

- 참고
    - http://bcho.tistory.com/1010
    - http://unlimitedpower.tistory.com/entry/NLP-Naive-Bayesian-Classification나이브-베이즈-분류
    - http://jihoonlee.tistory.com/22
    
한땀 한땀 코드를 작성하면서, 알고리즘을 이해해보도록 하겠습니다.

In [1]:
import pandas as pd
import numpy as np

In [2]:
# 손글씨 데이터는 mnist_train.csv 와 mnist_test.csv 로 이루어져 있습니다.
# Github에 jupyter notebook을 올려 공유하기 위해 용량이 작은 샘플 데이터로 진행하겠습니다.
# 참고로 저의 경우, 로컬에서는 Full Data 에서 샘플링하여 진행했습니다.

with open("./mnist_train_100.csv","r") as csvfile:
    train_data_raw = pd.read_csv(csvfile, header=None)
    
with open("./mnist_test_10.csv","r") as csvfile:
    test_data_raw = pd.read_csv(csvfile, header=None)

In [3]:
# 불러온 데이터를 살펴보겠습니다.

train_data_raw.info()
test_data_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Columns: 785 entries, 0 to 784
dtypes: int64(785)
memory usage: 613.4 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Columns: 785 entries, 0 to 784
dtypes: int64(785)
memory usage: 61.4 KB


In [4]:
# 학습데이터는 총 100개, 테스트데이터는 총 10개로 이루어져 있으며, 컬럼은 총 785개입니다.

test_data_raw.head(4)

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,784
0,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
test_data_raw.head(4)

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,784
0,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# 학습데이터와 테스트테이터 모두 첫번째 컬럼은 라벨이고 두번째 컬럼 부터 785번째 컬럼까지가 변수로 이루어져 있는 걸 확인할 수 있습니다.
# 분석상 용이를 위해 각 데이터셋에서 첫번째 컬럼은 라벨로 두번째 컬럼부터는 변수 데이터로 분리해 저장합니다.

test_labels = test_data_raw.iloc[:,0]
test_data = test_data_raw.iloc[:,1:]
train_labels = train_data_raw.iloc[:,0]
train_data = train_data_raw.iloc[:,1:]

먼저, **Naive Bayes 알고리즘**을 쉽게 이해하기 위해, 알고리즘을 부분적으로 나눠서 한단계 씩 따라가보도록 하겠습니다.

**Naive Bayes 알고리즘**에서 1번째 test data의 추정값은 1번째 데이터 셋이 주어졌을 때, 0부터 9까지 라벨이 발생할 확률을 각각 구한 뒤, 가장 확률이 높은 라벨을 선택하게 됩니다. 

1. 1번 test_data가 주어졌을 때, 해당 라벨이 0일 확률
    - P(0|test_data[0]) = P(test_data|0)∙P(0) / P(test_data[0])
2. 1번 test_data가 주어졌을 때, 해당 라벨이 1일 확률
    - P(1|test_data[0]) = P(test_data|1)∙P(1) / P(test_data[0])
3. 1번 test_data가 주어졌을 때, 해당 라벨이 2일 확률
    - P(2|test_data[0]) = P(test_data|2)∙P(2) / P(test_data[0])

...

10. 1번 test_data가 주어졌을 때, 해당 라벨이 9일 확률
    - P(9|test_data[0]) = P(test_data|9)∙P(9) / P(test_data[0])

우리는 각각의 확률의 크기만 비교하면 되기 때문에 동일한 값인 분모 P(test_data[0])를 모두 제외하고 값을 구한 뒤 비교를 하도록 하겠습니다.

그럼 우리는 다음의 값들만 구해서 그 확률의 크기만을 비교하면 됩니다.

- P(0|test_data[0]) = P(test_data|0)∙P(0) 
- P(1|test_data[0]) = P(test_data|1)∙P(1) 
- P(2|test_data[0]) = P(test_data|2)∙P(2) 

... 

- P(9|test_data[0]) = P(test_data|9)∙P(2) 


P(test_data|0) 은 라벨이 0일 때, test_data의 각각의 변수가 나타날 확률을 의미합니다.

우리에게 주어진 데이터는 784개의 변수값을 가지고 있는데, 만일 이 784개의 항목이 모두 독립이라고 가정하고 1번째 테스트 데이터가 1번 3번 10번 600번 변수가 0보다 크고 나머지는 다 0이라고 한다면, P(test_data|0)은 다음과 같이 표현할 수 있습니다.

- **P(test_data[0]|0) = P(1번변수|0)∙P(3번변수|0)∙P(10번변수|0)∙P(600번변수|0)**

![NB2_img](./img/p_2.png)

이제 실제 데이터에 위의 확률을 구해보도록 하겠습니다.

**P(0|test_data[0]) = P(test_data[0]|0)∙P(0) = P(1번변수|0)∙P(3번변수|0)∙P(10번변수|0)∙P(600번변수|0)∙P(0)** 

위 식에서 먼저 P(0)를 구해보도록 하겠습니다. (1번, 3번, 10번, 600번 변수가 0이 아니라는 것은 이해를 돕기 위한 예이며, 실제 1번째 데이터셋의 값은 다릅니다.)

**P(0) = count(라벨이 0인 갯수) / count(전체학습데이터 갯수)** 입니다.

In [7]:
train_data.shape[0] # 전체 학습데이터의 갯수는 100입니다.

100

In [8]:
T_dic = dict(train_labels.value_counts()) # 학습데이터 라벨이 각각 몇 개씩인지 세어봅니다. 센 결과값을 파이썬 딕셔너리 형식으로 저장해둡니다.
T_dic # 라벨이 0인 갯수는 13개 이네요

{0: 13, 1: 14, 2: 6, 3: 11, 4: 11, 5: 5, 6: 11, 7: 10, 8: 8, 9: 11}

In [9]:
# P(0) = 13 /100 
T_dic[0]/train_data.shape[0]

0.13

다음으로는 P(test_data[0]|0) 을 구해보도록 하겠습니다.

P(test_data[0]|0) 은 라벨이 0인 것들 중에서 각각의 항목값이 나타날 확률들의 곱을 구하면 됩니다.

In [10]:
train_data[train_data_raw[0] == 0].iloc[:,255:300]

,256,257,258,259,260,261,262,263,264,265,...,291,292,293,294,295,296,297,298,299,300
1,0,0,0,0,0,0,0,51,238,253,...,238,252,252,179,12,75,121,21,0,0
21,0,0,0,0,0,0,0,92,253,253,...,236,251,243,220,233,251,251,243,82,96
34,0,0,0,0,0,0,0,0,0,166,...,73,253,253,253,253,130,0,0,110,253
37,0,0,0,0,0,0,0,0,145,253,...,0,144,251,251,251,253,168,107,169,251
51,0,0,0,0,0,99,253,253,253,253,...,253,253,253,131,97,169,253,93,99,253
56,0,0,0,0,14,238,253,253,253,253,...,253,253,253,248,134,0,18,83,237,253
63,0,0,0,0,0,0,0,144,251,251,...,253,253,253,253,255,253,253,253,253,255
68,0,0,0,226,255,170,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
69,0,0,0,0,0,0,254,252,252,252,...,190,250,250,252,250,250,169,171,250,250
75,0,0,0,0,0,0,0,0,0,156,...,0,88,238,253,253,253,221,253,253,253


1번째 테스트 데이터의 784개의 항목 중에 0이 아닌 항목이 어떤 것인지 확인합니다.

In [11]:
test_x = test_data.iloc[0]
col_not_zero = test_x[test_x > 0].index 
col_not_zero # 203, 204, 205, 206, 207, 208, 231, 232, 233, 234 등이 0보다 큽니다.

Int64Index([203, 204, 205, 206, 207, 208, 231, 232, 233, 234,
            ...
            687, 711, 712, 713, 714, 715, 739, 740, 741, 742],
           dtype='int64', length=116)

- **P(test_data1|0) = P(203|0)∙P(204|0)∙P(205|0)∙∙∙P(742|0)**

In [12]:
Xs_label_i = train_data[train_data_raw[0] == 0].iloc[:,:] # 라벨이 0인 학습 데이터를 가져옵니다.
x_label_i = Xs_label_i[203]
x_label_i #학습데이터에서 라벨이 0인 것들 중 203번째 컬럼의 값들입니다.

1     0
21    0
34    0
37    0
51    0
56    0
63    0
68    0
69    0
75    0
81    0
88    0
95    0
Name: 203, dtype: int64

In [13]:
x_label_i[x_label_i>0].count() # P(203|0) 를 구하기 위해 0이 아닌 것들의 갯수를 세어봅니다. 

0

- **P(210|0) = count(라벨이 0인 것들 중 203번째 컬럼이 0이 아닌 것들) / count(라벨이 0인 것들의 전체 데이터 중 0이 아닌 것들)**

In [14]:
Xs_label_i[Xs_label_i>0].count().sum() # count(라벨이 0인 것들의 전체 데이터 중 0이 아닌 것들)

2585

In [15]:
P = x_label_i[x_label_i>0].count() / Xs_label_i[Xs_label_i>0].count().sum()
P

0.0

자, 이제 P(203|0) 를 구했으니, 나머지 P(204|0)∙P(205|0)∙∙∙P(742|0) 들을 반복해서 구해봅니다

In [16]:
Ps = []

for i in col_not_zero: # 계산은 210 부터 740 까지 반복합니다.

    x_label_i = Xs_label_i[i]

    P = x_label_i[x_label_i>0].count() / Xs_label_i[Xs_label_i>0].count().sum()
    Ps.append(P) # Ps 라는 변수에 210 부터 740까지 구한 확률을 넣어둡니다.

Ps

[0.0,
 0.00038684719535783365,
 0.00077369439071566729,
 0.0011605415860735009,
 0.0019342359767891683,
 0.0034816247582205029,
 0.00038684719535783365,
 0.00077369439071566729,
 0.0015473887814313346,
 0.0019342359767891683,
 0.0027079303675048355,
 0.0034816247582205029,
 0.0042553191489361703,
 0.0046421663442940036,
 0.0046421663442940036,
 0.0046421663442940036,
 0.0050290135396518377,
 0.0050290135396518377,
 0.0046421663442940036,
 0.0050290135396518377,
 0.0050290135396518377,
 0.0038684719535783366,
 0.00038684719535783365,
 0.0011605415860735009,
 0.0015473887814313346,
 0.0023210831721470018,
 0.0034816247582205029,
 0.0038684719535783366,
 0.0046421663442940036,
 0.0046421663442940036,
 0.0046421663442940036,
 0.0042553191489361703,
 0.0046421663442940036,
 0.0050290135396518377,
 0.0046421663442940036,
 0.0046421663442940036,
 0.0050290135396518377,
 0.0038684719535783366,
 0.0042553191489361703,
 0.0046421663442940036,
 0.0046421663442940036,
 0.0046421663442940036,
 0.00

다시 올라가서 보면, 저희는 1번째 테스트 데이터의 라벨이 0일 확률을 구하고 있었습니다.
1번째 테스트 데이터의 라벨이 0일 확률은 다음과 같이 정리되었습니다.

**P(0|test_data[0]) = P(203|0)∙P(204|0)∙P(205|0)∙∙∙P(742|0)∙P(0)**

즉, 위에서 구한 확률들을 곱하기만 하면 됩니다. 하지만 두가지 문제가 발생합니다.
**1. 만일 학습데이터가 값이 존재하지 않았던 값이 테스트 데이터에서 나왔다면, 그 확률 P(n|0) 는 0이 되어, 확률들을 곱한 값은 무조건 모두 0이 되어버립니다.**
**2. 확률들을 계속 반복해서 곱하다 보면, 값은 계속 0으로 수렴하고 매우 작은 수가 될 경우, 파이썬은 이를 0으로 처리합니다.**

먼저 첫번째 문제를 해결하기 위해서, P(203|0),P(204|0),P(742|0) 등의 값을 구할 때 **분자에 1을 모두 더해서** 확률값이 0이 되는 것을 방지하도록 하겠습니다. (이를 **Laplace Smoothing** 이라고 합니다.)

또한 두번째 문제를 해결하기 위해서, **계산식 양쪽에 log를 씌워 계산**하도록 하겠습니다. 저희가 궁금한 것은 값의 크기 비교이지 정확한 값이 아니니깐요.

위의 수정된 조건을 반영해서 1번째 테스트 데이터의 라벨이 0일 확률 부터 9일 확률을 구한 뒤, 그 값이 가장 큰 값을 1번 테스트 데이터의 라벨로 예측합니다. 이 과정을 1번째 테스트 데이터부터 10번째 테스트 데이터까지 반복하는 파이썬 코드를 작성하면 다음과 같습니다.

In [17]:
T = train_labels.value_counts()
T_dic = dict(T)
T_dic

test_est_fs = []

for t in range(len(test_data)):
    test_x = test_data.iloc[t]
    col_not_zero = test_x[test_x > 0].index
    log_P_i = []
    log_P_d = []

    for k in range(0,10): # 라벨값을 0부터 9까지 구합니다.
        
        Xs_label_i = train_data[train_data_raw[0] == k].iloc[:,:] # 라벨이 k인 학습 데이터를 가져옵니다.

        Ps =[]

        for i in col_not_zero: # 테스트 데이터가 가지고 있는 컬럼값들의 확률만 구하여 계산합니다.
            x_label_i = Xs_label_i[i]
           
            p = (x_label_i[x_label_i>0].count()+1) / (Xs_label_i[Xs_label_i>0].count().sum() + 784) # 학습데이터에 있지 않던 데이터가 등장해 확률 곱이 0이 되는 것을 방지하기 위해 분자, 분모에 1과 유니크한 데이터값의 객수를 더해줍니다.
            Ps.append(p)
            
        log_P = np.log(Ps).sum() + np.log(T[train_labels[0]] / train_data.shape[0]) # 확률의 곱이 0에 수렴하는 것을 방지하기 위해 확률에 log를 씌워 값을 구합니다.
        log_P_i.append(k)
        log_P_d.append(log_P)

    test_est = []
    test_est = pd.DataFrame(test_est)

    test_est["labels"] = log_P_i
    test_est["p"] = log_P_d

    test_est.sort_values(['p'], ascending=False, inplace=True) # 발생할 확률값이 가장 큰 값을 구하기 위해 내림차순으로 값을 정리합니다.
    test_est_f = test_est[:1]['labels'].iloc[0] # 발생할 확률이 가장 높은 것을 예측값으로 선택합니다.
    test_est_fs.append(test_est_f)
    print("%d 번째 테스트 데이터 예측이 완료되었습니다. 예측값은 %d 입니다." %(t+1, test_est_f))
    

1 번째 테스트 데이터 예측이 완료되었습니다. 예측값은 7 입니다.
2 번째 테스트 데이터 예측이 완료되었습니다. 예측값은 2 입니다.
3 번째 테스트 데이터 예측이 완료되었습니다. 예측값은 1 입니다.
4 번째 테스트 데이터 예측이 완료되었습니다. 예측값은 0 입니다.
5 번째 테스트 데이터 예측이 완료되었습니다. 예측값은 4 입니다.
6 번째 테스트 데이터 예측이 완료되었습니다. 예측값은 1 입니다.
7 번째 테스트 데이터 예측이 완료되었습니다. 예측값은 4 입니다.
8 번째 테스트 데이터 예측이 완료되었습니다. 예측값은 4 입니다.
9 번째 테스트 데이터 예측이 완료되었습니다. 예측값은 4 입니다.
10 번째 테스트 데이터 예측이 완료되었습니다. 예측값은 9 입니다.


In [18]:
# 정확도 비교를 용이하게 하기 위해 테스트 데이터의 실제 라벨과 예측한 라벨을 하나의 데이터프레임에 저장합니다.
test_rst = []
test_rst = pd.DataFrame(test_rst)
test_rst["test_labels"] = test_labels
test_rst["estimated_labels"] = test_est_fs
test_rst

,test_labels,estimated_labels
0,7,7
1,2,2
2,1,1
3,0,0
4,4,4
5,1,1
6,4,4
7,9,4
8,5,4
9,9,9


In [19]:
# 정확도를 계산합니다.
correct = 0
for p in range(len(test_est)):
    if test_rst["test_labels"][p] == test_rst["estimated_labels"][p]:
        correct += 1 # 실제 라벨과 예측 라벨이 같은 것의 갯수를 셉니다.
            
accuracy = (correct/float(len(test_rst))) * 100.0 # 맞춘 갯수를 전체 갯수로 나누어 정확도를 구합니다.     

print("accuracy=%.2f%%" % (accuracy))

accuracy=80.00%


이번에는 python library 중 **sikit-learn** 에 있는 **KNeighborsClassifier** 를 가지고 손쉽게 kNN 알고리즘 결과를 구해보도록 하겠습니다.

In [20]:
from sklearn import datasets
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(train_data, train_labels)

# evaluate the model and update the accuracies list
score = model.score(test_data, test_labels)

print("accuracy=%.2f%%" % (score * 100))
predictions = model.predict(test_data)

test_est = []
test_est = pd.DataFrame(test_est)
test_est["test_labels"] = test_labels
test_est["estimated_labels"] = predictions

accuracy=80.00%


In [21]:
test_est

,test_labels,estimated_labels
0,7,7
1,2,2
2,1,1
3,0,0
4,4,4
5,1,1
6,4,4
7,9,4
8,5,4
9,9,9
